In [1]:
import pandas as pd
import json
import json
from tqdm import tqdm
import pandas as pd
import sys
sys.path.append("../src")
import prompt_utils
import os
import random
from typing import List

# vicuna 
# with rules classification only (beta 0.5: 0.76, beta 0.25: 0.86) (0.7-0.8s per prompt)
vicuna_base_path = "../data/vicuna_4bit/"
vicuna_with_rules_classification_only_name = "generic_prompt_with_rules_only_classification"
vicuna_with_rules_classification_only_func = prompt_utils.get_vicuna_prompt_with_rules_only_classification

# OA LLAMA
# Classification Only V03 (bta 0.5: 0.81, beta 0.25: 0.83)
# With Rules Classification only (beta 0.5: 0.8, beta 0.25: 0.89)
# With 3 Random Examples Classification only (beta 0.5: 0.78, beta 0.25: 0.88)
oa_base_path = "../data/openassistant_llama_30b_4bit/"
oa_classification_only_v03_name = "generic_prompt_without_context_only_classification_v03"
oa_classification_only_v03_func = prompt_utils.get_openassistant_llama_30b_4bit_without_context_only_classification_v03 #
oa_with_rules_classification_only_name = "generic_prompt_with_rules_only_classification"
oa_with_rules_classification_only_func = prompt_utils.get_openassistant_llama_30b_4bit_with_rules_only_classification
oa_with_3_random_examples_classification_only_name = "generic_prompt_few_shot_prompt_only_classification_3_random_example"
oa_with_rules_classification_only_func = prompt_utils.get_openassistant_llama_30b_4bit_few_shot_prompt_only_classification_n_random_example

# Text Davinci
# Elaboration First V04 (beta 0.5: 0.87, beta 0.25: 0.93)
davinci_base_path = "../data/openai_text_davinci_003/"
davinci_elaboration_first_v04_name = "generic_prompt_without_context_elaboration_first_v04"
davinci_elaboration_first_v04_func = prompt_utils.get_openai_prompt_without_context_elaboration_first_v04

# Define a list of filenames to load
labeled_data_filename = "../data/labeled_data/generic_test_0.json"

dfs = []
with open(labeled_data_filename) as f:
    data = json.load(f)
df = pd.DataFrame(data["train"])
dfs.append(df)
df = pd.DataFrame(data["test"])
dfs.append(df)
df = pd.DataFrame(data["valid"])
dfs.append(df)
df_all = pd.concat(dfs)
ALL_LABELS = prompt_utils.ALL_LABELS[:-1]
LOW_F1_LABELS = prompt_utils.LOW_F1_LABELS

In [2]:
def initialize_eng_dataframe():
    unlabeled_dataset = prompt_utils.generate_unlabeled_dataset()
    return unlabeled_dataset.loc[unlabeled_dataset.tweet_language == "en"].sample(frac=1, random_state = 42).reset_index()

def find_start_index(df: pd.DataFrame, new_column_names: List[str]) -> int:
    # Find first row where all weak labels are NaN
    try:
        start_index = df[new_column_names].isna().all(axis=1).idxmax()
        # If no NaN found in any of new_column_names, start from beginning
        if pd.isna(start_index):
            start_index = 0
        return start_index
    except:
        return 0

In [3]:
output_folder = f"{vicuna_base_path}/{vicuna_with_rules_classification_only_name}"
model_name = "low_f1_labels_weak_labeling"
prompt_func = vicuna_with_rules_classification_only_func
idx_of_rules_of_low_f1_labels = [1, 2, 4, 9, 12, 13]

if os.path.isfile(f"{output_folder}/{model_name}.csv"):
    eng_tweets = pd.read_csv(f"{output_folder}/{model_name}.csv")
else:
    eng_tweets = initialize_eng_dataframe()
    eng_tweets.to_csv(f"{output_folder}/{model_name}.csv", index=False)

/tmp/ipykernel_8042/3980851207.py:7: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,19,20,22,23,28,29,30,32,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  eng_tweets = pd.read_csv(f"{output_folder}/{model_name}.csv")


In [4]:
LOW_F1_LABELS_COLUMNS = [i + "_pred" for i in LOW_F1_LABELS]
start_index = find_start_index(eng_tweets, LOW_F1_LABELS_COLUMNS)

In [5]:
start_index

18106

In [6]:
import llm_utils
eng_tweets = eng_tweets.iloc[0:18105]
for label in LOW_F1_LABELS:
    eng_tweets[f'{label}_pred'] = eng_tweets[f'{label}_pred'].apply(llm_utils.get_extraction_function("extract_using_class_token", 1))

In [7]:
import numpy as np
mask_conspiracy = eng_tweets['Conspiracy Theory_pred'] == 0
mask_education = eng_tweets['Education_pred'] == 0
mask_environment = eng_tweets['Environment_pred'] == 0
mask_labor = eng_tweets['Labor/Employment_pred'] == 0
mask_religion = eng_tweets['Religion_pred'] == 0
mask_science = eng_tweets['Science/Technology_pred'] == 0

conspiracy_preds = eng_tweets[eng_tweets['Conspiracy Theory_pred'] == 1]
print("Length Conspiracy Theory predictions: ", len(conspiracy_preds))

only_conspiracy_predicted = eng_tweets[(eng_tweets['Conspiracy Theory_pred'] == 1) & (mask_education & mask_environment & mask_labor & mask_religion & mask_science)]
print("Length Conspiracy Theory predictions with all other predictions being 0: ", len(only_conspiracy_predicted))

education_preds = eng_tweets[eng_tweets['Education_pred'] == 1]
print("Length Education predictions: ", len(education_preds))

only_education_predicted = eng_tweets[eng_tweets['Education_pred'] == 1 & mask_conspiracy & mask_environment & mask_labor & mask_religion & mask_science]
print("Length Education predictions with all other predictions being 0: ", len(only_education_predicted))

environment_preds = eng_tweets[eng_tweets['Environment_pred'] == 1]
print("Length Environment predictions: ", len(environment_preds))

only_environment_predicted = eng_tweets[eng_tweets['Environment_pred'] == 1 & mask_conspiracy & mask_education & mask_labor & mask_religion & mask_science]
print("Length Environment predictions with all other predictions being 0: ", len(only_environment_predicted))

labor_preds = eng_tweets[eng_tweets['Labor/Employment_pred'] == 1]
print("Length Labor/Employment predictions: ", len(labor_preds))

only_labor_predicted = eng_tweets[eng_tweets['Labor/Employment_pred'] == 1 & mask_conspiracy & mask_education & mask_environment & mask_religion & mask_science]
print("Length Labor/Employment predictions with all other predictions being 0: ", len(only_labor_predicted))

religion_pred = eng_tweets[eng_tweets['Religion_pred'] == 1]
print("Length Religion predictions: ", len(religion_pred))

only_religion_predicted = eng_tweets[eng_tweets['Religion_pred'] == 1 & mask_conspiracy & mask_education & mask_environment & mask_labor & mask_science]
print("Length Religion predictions with all other predictions being 0: ", len(only_religion_predicted))

science_pred = eng_tweets[eng_tweets['Science/Technology_pred'] == 1]
print("Length Science/Technology predictions: ", len(science_pred))

only_science_predicted = eng_tweets[eng_tweets['Science/Technology_pred'] == 1 & mask_conspiracy & mask_education & mask_environment & mask_labor & mask_religion]
print("Length Science/Technology predictions: ", len(only_science_predicted))

science_pred[["tweet_text", "Conspiracy Theory_pred", "Education_pred", "Environment_pred", "Labor/Employment_pred", "Religion_pred", "Science/Technology_pred"]]

Length Conspiracy Theory predictions:  962
Length Conspiracy Theory predictions with all other predictions being 0:  879
Length Education predictions:  778
Length Education predictions with all other predictions being 0:  2557
Length Environment predictions:  933
Length Environment predictions with all other predictions being 0:  2579
Length Labor/Employment predictions:  699
Length Labor/Employment predictions with all other predictions being 0:  2627
Length Religion predictions:  348
Length Religion predictions with all other predictions being 0:  2914
Length Science/Technology predictions:  290
Length Science/Technology predictions:  2842


,tweet_text,Conspiracy Theory_pred,Education_pred,Environment_pred,Labor/Employment_pred,Religion_pred,Science/Technology_pred
106,The #Angara #rocket is built on a modular desi...,0.0,0.0,0.0,0.0,0.0,1.0
154,RT @djnicknicholas: On #TheHitLab tomorrow @Ka...,0.0,0.0,0.0,0.0,0.0,1.0
246,"VIDEO : OneWeb Founder on Bringing 3,000 Jobs ...",0.0,1.0,0.0,1.0,0.0,1.0
345,RT @NRMOnline: President @KagutaMuseveni comme...,0.0,1.0,0.0,0.0,0.0,1.0
373,RT @CQvMyyB0YfvwrUrsaZ6KI7yqaJfSUDTrAI0joQhgMA...,0.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...
17764,Giuliani: Trump to fight hacking with cybersec...,0.0,0.0,0.0,0.0,0.0,1.0
17815,RT @TButaka: Which Definition can you never fo...,0.0,0.0,0.0,0.0,0.0,1.0
17867,This could prevent distracted driving https://...,0.0,1.0,1.0,0.0,0.0,1.0
17893,Wireless headphones for the holidays https://t...,0.0,0.0,0.0,0.0,0.0,1.0


In [36]:
LOW_F1_LABELS_COLUMNS = [i + "_pred" for i in LOW_F1_LABELS]
start_index = find_start_index(eng_tweets, LOW_F1_LABELS_COLUMNS)
start_index = 4143
print("Starting at index:", start_index)
for i in tqdm(range(start_index, len(eng_tweets)), total = len(eng_tweets)-start_index):
    for label, rule_idx in zip(LOW_F1_LABELS, idx_of_rules_of_low_f1_labels):
        try:
            new_column_name = f'{label}_pred'
            tweet_text = prompt_utils.normalize_tweet_simplified(eng_tweets.iloc[i]['tweet_text'])
            #print(tweet_text)
            prompt, followup, request_params = prompt_func(tweet_text, label, prompt_utils.RULES[rule_idx], prompt_utils.get_base_request_params())
            response = prompt_utils.get_response(request_params, prompt, "")
            # for openai
            #response = prompt_utils.get_response_wip(prompt, "openai-davinci", max_tokens = 400, openai_model = "davinci")
            eng_tweets.loc[i, new_column_name] = response
        except Exception as e:
            print(e)
            print("Error at index:", i)
            print("Label:", label)
            eng_tweets.loc[i, new_column_name] = "ERROR"
    
    if i % 300 == 0:
        eng_tweets.to_csv(f"{output_folder}/{model_name}.csv", index=False)
        print("Saved at index:", i)
        print("Last label:", label)
        print("Last response:", response)

eng_tweets.to_csv(f"{output_folder}/{model_name}.csv", index=False)

Starting at index: 4143


  0%|          | 0/1565961 [00:00<?, ?it/s]

'float' object has no attribute 'replace'
Error at index: 4143
Label: Conspiracy Theory
'float' object has no attribute 'replace'
Error at index: 4143
Label: Education
'float' object has no attribute 'replace'
Error at index: 4143
Label: Environment
'float' object has no attribute 'replace'
Error at index: 4143
Label: Labor/Employment
'float' object has no attribute 'replace'
Error at index: 4143
Label: Religion
'float' object has no attribute 'replace'
Error at index: 4143
Label: Science/Technology
'float' object has no attribute 'replace'
Error at index: 4144
Label: Conspiracy Theory
'float' object has no attribute 'replace'
Error at index: 4144
Label: Education
'float' object has no attribute 'replace'
Error at index: 4144
Label: Environment
'float' object has no attribute 'replace'
Error at index: 4144
Label: Labor/Employment
'float' object has no attribute 'replace'
Error at index: 4144
Label: Religion
'float' object has no attribute 'replace'
Error at index: 4144
Label: Science/T

  0%|          | 58/1565961 [04:42<4454:00:35, 10.24s/it]

Saved at index: 4200
Last label: Science/Technology
Last response:  0 (Not about Science/Technology


...

Saved at index: 17700
Last label: Science/Technology
Last response:  0 (Not about Science/Technology


  1%|          | 13858/1565961 [19:39:07<4776:43:20, 11.08s/it]

Saved at index: 18000
Last label: Science/Technology
Last response:  0 (Not about Science/Technology


  1%|          | 13962/1565961 [19:48:24<2201:42:19,  5.11s/it]


KeyboardInterrupt: 

In [37]:
eng_tweets.to_csv(f"{output_folder}/{model_name}.csv", index=False)

In [35]:
eng_tweets.iloc[4280]

index                                                                  424685
tweetid                                                   1289113606794223616
userid                                                    1108056700450414592
user_display_name                                                Hustle Queen
user_screen_name                                                HustleQueenUg
user_reported_location                                                 Uganda
user_profile_description                      patriot of the great Sevolution
user_profile_url                                                          NaN
follower_count                                                        13491.0
following_count                                                          5975
account_creation_date                                              2019-03-19
account_language                                                           en
tweet_language                                                  

In [26]:
eng_tweets.to_csv(f"{output_folder}/{model_name}.csv", index=False)